# Process the data obtained from polarimetry and reorganize the data into the correct folders

In [18]:
%load_ext autoreload
%autoreload 2

import os
from processingmm import batch_processing, libmpMuelMat

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
# set the parameters to run the script
directories = [r'E:\Chicago\pat1-sec1-denoised']
calib_directory = r'C:\Users\romai\Documents\program_files\reconstructor\calibration'

In [20]:
# set run_all to true in order to run the pipeline on all the folders (even the ones already processed)
# set the parameters to be used for the line visualisation with the parameter parameter_set
batch_processing.batch_process(directories, calib_directory, run_all = True, parameter_set = 'GCB_symposium_Fixed')

Traceback (most recent call last):
  File "C:\Users\romai\Documents\processingMM\src\processingmm\batch_processing.py", line 70, in find_folder_name
    assert len(re.findall("[\d]{4}-[\d]{2}-[\d]{2}", root)) == 1
AssertionError
100%|██████████████████████████████████████████████████████████████████████████████████| 21/21 [16:41<00:00, 47.70s/it]


['E:\\Chicago\\pat1-sec1-denoised\\2023-08-29_T_PAT1-SEC1-C6_FX_M_1',
 'E:\\Chicago\\pat1-sec1-denoised\\2023-08-29_T_PAT1-SEC1-B3_FX_M_1',
 'E:\\Chicago\\pat1-sec1-denoised\\2023-08-29_T_PAT1-SEC1-C2_FX_M_1',
 'E:\\Chicago\\pat1-sec1-denoised\\2023-08-30_T_PAT1-SEC1-X3_FX_M_1',
 'E:\\Chicago\\pat1-sec1-denoised\\2023-08-29_T_PAT1-SEC1-F1_FX_M_1',
 'E:\\Chicago\\pat1-sec1-denoised\\2023-08-30_T_PAT1-SEC1-X4_FX_M_1',
 'E:\\Chicago\\pat1-sec1-denoised\\2023-08-29_T_PAT1-SEC1-B1_FX_M_1',
 'E:\\Chicago\\pat1-sec1-denoised\\2023-08-29_T_PAT1-SEC1-D4_FX_M_1',
 'E:\\Chicago\\pat1-sec1-denoised\\2023-08-29_T_PAT1-SEC1-G3_FX_M_1',
 'E:\\Chicago\\pat1-sec1-denoised\\2023-08-29_T_PAT1-SEC1-F3_FX_M_1',
 'E:\\Chicago\\pat1-sec1-denoised\\2023-08-29_T_PAT1-SEC1-B4_FX_M_1',
 'E:\\Chicago\\pat1-sec1-denoised\\2023-08-30_T_PAT1-SEC1-X5_FX_M_1',
 'E:\\Chicago\\pat1-sec1-denoised\\2023-08-29_T_PAT1-SEC1-A3_FX_M_1',
 'E:\\Chicago\\pat1-sec1-denoised\\2023-08-29_T_PAT1-SEC1-G1_FX_M_1',
 'E:\\Chicago\\pat1-